In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Hugging face login with token


In [ ]:
from huggingface_hub import login
login()

Unzip

In [ ]:

import zipfile
import os

def unzip_folder(zip_file_path, extract_path):
  """Unzips a zip file to a specified location.

  Args:
    zip_file_path: The path to the zip file.
    extract_path: The directory to extract the zip file contents to.
  """
  try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)
    print(f"Successfully unzipped '{zip_file_path}' to '{extract_path}'")
  except FileNotFoundError:
    print(f"Error: Zip file '{zip_file_path}' not found.")
  except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


zip_file = "/content/drive/MyDrive/archive.zip"
extract_dir = "/content/drive/MyDrive/Spider"

os.makedirs(extract_dir, exist_ok=True)

unzip_folder(zip_file, extract_dir)


Successfully unzipped '/content/drive/MyDrive/archive.zip' to '/content/drive/MyDrive/Spider'


In [ ]:
import pandas as pd

In [ ]:

df1 = pd.read_json("/content/drive/MyDrive/Spider/spider/train_spider.json")



In [ ]:

df2 = pd.read_json("/content/drive/MyDrive/Spider/spider/train_others.json")



In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
len(df)

8659

In [ ]:
df.head(10)

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'except': None, 'from': {'conds': [], 'table_..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'except': None, 'from': {'conds': [], 'table_..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'except': None, 'from': {'conds': [], 'table_..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'except': None, 'from': {'conds': [], 'table_..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'except': None, 'from': {'conds': [], 'table_..."
5,department_management,SELECT name FROM head WHERE born_state != 'Cal...,"[SELECT, name, FROM, head, WHERE, born_state, ...","[select, name, from, head, where, born_state, ...",What are the names of the heads who are born o...,"[What, are, the, names, of, the, heads, who, a...","{'except': None, 'from': {'conds': [], 'table_..."
6,department_management,SELECT DISTINCT T1.creation FROM department AS...,"[SELECT, DISTINCT, T1.creation, FROM, departme...","[select, distinct, t1, ., creation, from, depa...",What are the distinct creation years of the de...,"[What, are, the, distinct, creation, years, of...","{'except': None, 'from': {'conds': [[False, 2,..."
7,department_management,SELECT born_state FROM head GROUP BY born_stat...,"[SELECT, born_state, FROM, head, GROUP, BY, bo...","[select, born_state, from, head, group, by, bo...",What are the names of the states where at leas...,"[What, are, the, names, of, the, states, where...","{'except': None, 'from': {'conds': [], 'table_..."
8,department_management,SELECT creation FROM department GROUP BY creat...,"[SELECT, creation, FROM, department, GROUP, BY...","[select, creation, from, department, group, by...",In which year were most departments established?,"[In, which, year, were, most, departments, est...","{'except': None, 'from': {'conds': [], 'table_..."
9,department_management,"SELECT T1.name , T1.num_employees FROM depart...","[SELECT, T1.name, ,, T1.num_employees, FROM, d...","[select, t1, ., name, ,, t1, ., num_employees,...",Show the name and number of employees for the ...,"[Show, the, name, and, number, of, employees, ...","{'except': None, 'from': {'conds': [[False, 2,..."


In [ ]:
df.drop(df.columns[[2,3,5,6]], axis=1, inplace=True)

In [ ]:
df.head(10)

,db_id,query,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
5,department_management,SELECT name FROM head WHERE born_state != 'Cal...,What are the names of the heads who are born o...
6,department_management,SELECT DISTINCT T1.creation FROM department AS...,What are the distinct creation years of the de...
7,department_management,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...
8,department_management,SELECT creation FROM department GROUP BY creat...,In which year were most departments established?
9,department_management,"SELECT T1.name , T1.num_employees FROM depart...",Show the name and number of employees for the ...


In [ ]:

from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.8, random_state=42)


print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Testing set size:", len(test_df))


In [ ]:
import os

base_directory = '/content/drive/MyDrive/Spider/spider/database'
def get_schema_for_db(db_id):

    schema_file_path = os.path.join(base_directory, db_id, 'schema.sql')
    if os.path.exists(schema_file_path):
        with open(schema_file_path, 'r') as file:
            for line in file:
                if "CREATE" in line.upper():
                    create_lines.append(line)
        return create_lines
    else:
        return None

df['schema'] = df['db_id'].apply(get_schema_for_db)


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
df = df.reset_index(drop=True)


In [ ]:
from datasets import DatasetDict
fin_data=DatasetDict({
    "train":train_dataset,
    "test":test_dataset,
    "val":val_dataset
},)

In [ ]:
train_dataset[:100]

In [ ]:
from google.colab import drive
import zipfile
import os
import pandas as pd
from datasets import Dataset
from huggingface_hub import HfApi

api = HfApi(token="repo_token")

api.create_repo(repo_id="rakshithjoseph/spider_data", repo_type="dataset", exist_ok=True)
fin_data.push_to_hub("rakshithjoseph/spider_data")



Uploading the dataset shards:   0%|          | 0/15 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/689 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/rakshithjoseph/spider_data/commit/f3fd1f8eec45ed5c8e2741fd49d7418498b13e3a', commit_message='Upload dataset', commit_description='', oid='f3fd1f8eec45ed5c8e2741fd49d7418498b13e3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rakshithjoseph/spider_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rakshithjoseph/spider_data'), pr_revision=None, pr_num=None)